Observing the Data

To do the reprocessing, we first have to take a brief look on our data.

We are now looking for a. unneeded columns, b. missing values, c. duplicates

In [112]:
import numpy as np
import pandas as pd

data = pd.read_csv("E:/git/project/.venv/house_kg_10K_ads.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           10000 non-null  int64  
 1   m2_price        10000 non-null  int64  
 2   square          10000 non-null  int64  
 3   rooms           10000 non-null  object 
 4   district        9990 non-null   object 
 5   micro_district  9543 non-null   object 
 6   building_type   10000 non-null  object 
 7   floor           10000 non-null  int64  
 8   floors          10000 non-null  int64  
 9   year            5631 non-null   float64
 10  date            10000 non-null  object 
 11  source          10000 non-null  object 
 12  condition       7081 non-null   object 
dtypes: float64(1), int64(5), object(7)
memory usage: 1015.8+ KB


For the purpose of price predicting, some of the columns need to be removed from the dataset.

Yet they might do an impact to the price of the house, their overall influence to the dataset seems to be quite low, so it's not critical to get rid of them in order to make the program simple.

These columns are:
4. district, 5. micro_district, 6. building_type, as the influence of the house location and the material it was built of are already represented in the 1. m2_price.

9. year, as most of the houses provided in the dataset were built in the period from 2010 to 2022 and these difference does not influence the price much; also there are a lot of missing values in this column and it's quite problematic to solve this issue by simple imputation.

10. date and 11. source, as the date of the ad and the site at which it's placed are the irrelevant data for the purposes of our project.

In [113]:
data.drop(["district", "micro_district", "building_type", "year", "date", "source"], axis = 1, inplace = True)
data.head()

,price,m2_price,square,rooms,floor,floors,condition
0,110880,720,154,4,12,14,под самоотделку (ПСО)
1,57000,740,77,2,5,12,под самоотделку (ПСО)
2,57750,750,77,2,5,12,под самоотделку (ПСО)
3,57000,740,77,2,5,12,под самоотделку (ПСО)
4,55000,1279,43,1,1,12,хорошее


It's done with columns. Now we can deal with missing values and duplicates.


In [114]:
data.isna().sum()

price           0
m2_price        0
square          0
rooms           0
floor           0
floors          0
condition    2919
dtype: int64

There are no missing data in all the columns except the last one, which tell us the condition of the house. As this data is categorical, we'll deal with it later.

Now we drop all duplicates.

In [115]:
data.drop_duplicates(inplace = True)

Back to the categorical data. The only column that includes it is «condition».

We have to encode it. As the categories in this column can be easily ordered, we use Ordinal Encoding method. We set a correspondance between each category and some integer. The more the integer is, the better the condition of the house.

So, excluding missing data, we've got 8 categories.

1. «repair required», 
2. «not built yet», 
3. «rough finish», 
4. «free-pattern layout». 

We're going to drop the rows with this categories above as they are outliers and represent only 3% of dataset.

5. «average». This condition corresponds to number 1.
6. «good» — 2
7. «self-finishing» — 3
8. «euro-repair» — 4

In [116]:
data = data[~data['condition'].isin(["не достроено", "требует ремонта", "черновая отделка", "свободная планировка"])]

encoding = {'среднее': 1, 'хорошее': 2, 'под самоотделку (ПСО)': 3, 'евроремонт': 4}
data['condition_encoded'] = data['condition'].map(encoding)
data.head(5)

,price,m2_price,square,rooms,floor,floors,condition,condition_encoded
0,110880,720,154,4,12,14,под самоотделку (ПСО),3.0
1,57000,740,77,2,5,12,под самоотделку (ПСО),3.0
2,57750,750,77,2,5,12,под самоотделку (ПСО),3.0
4,55000,1279,43,1,1,12,хорошее,2.0
5,97000,1276,76,2,10,12,евроремонт,4.0


Now we can deal with the missing values. Unless the dataset is big itself /9000 rows/, I would not like to delete this big chunk of information. We can use some smart imputation.

Missing values in the «condition» column are 25% of the dataset. We are going to imput the values according to their natural distribution in the dataset: first, we'll calculate the probabilities of each «level» of condition to appear in the dataset; then we imput this values in this 25% rows with missing values accordingly with calculated probabilities.

At the end, we encode the imputated rows again.

In [117]:
probabilities = data['condition'].value_counts(normalize = True)
print(probabilities)
data['condition'] = data['condition'].apply(lambda x: np.random.choice(probabilities.index, p=probabilities.values) if pd.isna(x) else x)
data['condition_encoded'] = data['condition'].map(encoding)

data.isna().sum()
data.head(5)

condition
евроремонт               0.343099
хорошее                  0.287681
под самоотделку (ПСО)    0.279209
среднее                  0.090011
Name: proportion, dtype: float64


,price,m2_price,square,rooms,floor,floors,condition,condition_encoded
0,110880,720,154,4,12,14,под самоотделку (ПСО),3
1,57000,740,77,2,5,12,под самоотделку (ПСО),3
2,57750,750,77,2,5,12,под самоотделку (ПСО),3
4,55000,1279,43,1,1,12,хорошее,2
5,97000,1276,76,2,10,12,евроремонт,4


Now the data is reprocessed. Further manipulations with it should be done with the consideration of the machine learning model that is going to be used. That's why we do not dive into normalization yet.